# Fine-Tuning SmolLM3-3B for Kotlin Multiplatform Code Generation

**Optimized for Google Colab T4 GPU**

This notebook fine-tunes SmolLM3-3B on Kotlin KMP code with **full method body implementations**.

## Key Features:
- ✅ **QLoRA** for efficient training on T4 (15GB VRAM)
- ✅ **Method body validation** - Tests if model generates full implementations
- ✅ **Production-ready** - Gradient checkpointing, mixed precision
- ✅ **Quality metrics** - Evaluates implementation vs signature-only generation

## Dataset Stats:
- **132,577** training pairs
- **52.5%** have real method bodies
- **7 pair types**: expect/actual, interface→impl, description→code, etc.

---

## 1. Setup & Installation

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv

In [ ]:
%%capture
# Install dependencies
!pip install -q transformers==4.48.0 datasets==3.2.0 peft==0.14.0 bitsandbytes==0.45.0 trl==0.12.0 accelerate==1.2.1 sentencepiece protobuf

In [ ]:
import torch
import json
import os
from pathlib import Path
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import re

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

## 2. Mount Google Drive & Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# === CONFIGURE YOUR DATA PATH ===
# Upload your data/final_training/ folder to Google Drive first
DATA_DIR = "/content/drive/MyDrive/kmp_training_data/final_training"

# Verify files exist
!ls -lh {DATA_DIR}/*.jsonl

In [ ]:
# Load training data
def load_kmp_dataset(data_dir):
    """Load KMP training data from JSONL files"""
    
    train_data = []
    val_data = []
    
    # Load training pairs
    train_file = f"{data_dir}/train.jsonl"
    val_file = f"{data_dir}/val.jsonl"
    
    print(f"Loading training data from {train_file}...")
    with open(train_file, 'r') as f:
        for line in f:
            train_data.append(json.loads(line))
    
    print(f"Loading validation data from {val_file}...")
    with open(val_file, 'r') as f:
        for line in f:
            val_data.append(json.loads(line))
    
    print(f"\nDataset loaded:")
    print(f"  Train: {len(train_data):,} examples")
    print(f"  Val:   {len(val_data):,} examples")
    
    return Dataset.from_list(train_data), Dataset.from_list(val_data)

train_dataset, val_dataset = load_kmp_dataset(DATA_DIR)

In [ ]:
# Analyze dataset
import pandas as pd
from collections import Counter

# Get pair type distribution
pair_types = Counter([item['pair_type'] for item in train_dataset])

print("\n📊 Pair Type Distribution:")
for ptype, count in pair_types.most_common():
    pct = 100 * count / len(train_dataset)
    print(f"  {ptype:35s} {count:>7,}  ({pct:5.1f}%)")

# Sample examples
print("\n📝 Sample Training Example:")
sample = train_dataset[0]
print(f"\nPair Type: {sample.get('pair_type', 'unknown')}")
print(f"Source Set: {sample.get('source_set', 'unknown')}")
print(f"\nINPUT (first 200 chars):\n{sample['input_text'][:200]}...")
print(f"\nTARGET (first 300 chars):\n{sample['target_text'][:300]}...")

## 3. Load Model with QLoRA (T4 Optimized)

In [ ]:
# Model configuration
MODEL_NAME = "HuggingFaceTB/SmolLM2-1.7B-Instruct"  # Using 1.7B for T4 safety
# Alternative: "HuggingFaceTB/SmolLM3-3B" if you have more VRAM

# QLoRA configuration for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading model: {MODEL_NAME}")
print(f"Quantization: 4-bit NF4")

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Tokenizer loaded:")
print(f"  Vocab size: {len(tokenizer)}")
print(f"  EOS token: {tokenizer.eos_token}")
print(f"  PAD token: {tokenizer.pad_token}")

In [ ]:
# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Prepare for k-bit training
model = prepare_model_for_kbit_training(model)

print(f"\nModel loaded successfully!")
print(f"Memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")

## 4. Configure LoRA

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=16,                       # LoRA rank
    lora_alpha=32,              # LoRA alpha (scaling factor)
    target_modules=[            # Target attention modules
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## 5. Format Training Data

In [ ]:
def format_prompt(example):
    """
    Format training example into instruction format.
    
    Uses a clear instruction format that emphasizes FULL implementation.
    """
    input_text = example['input_text']
    target_text = example['target_text']
    
    # Create instruction-following format
    prompt = f"""<|im_start|>system
You are an expert Kotlin Multiplatform developer. Generate complete, working code with full method body implementations. Never output just signatures or TODO comments.<|im_end|>
<|im_start|>user
{input_text}<|im_end|>
<|im_start|>assistant
{target_text}<|im_end|>"""
    
    return {"text": prompt}

# Format datasets
train_dataset_formatted = train_dataset.map(format_prompt)
val_dataset_formatted = val_dataset.map(format_prompt)

# Show example
print("\n📝 Formatted Example (first 500 chars):")
print(train_dataset_formatted[0]['text'][:500])

## 6. Training Configuration

In [ ]:
# Training arguments optimized for T4
training_args = TrainingArguments(
    # Output
    output_dir="./smollm3-kmp-finetuned",
    
    # Training regime
    num_train_epochs=3,
    per_device_train_batch_size=2,      # Small batch for T4
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,       # Effective batch size = 16
    
    # Optimization
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    
    # Memory optimization
    gradient_checkpointing=True,
    bf16=True,                           # BFloat16 mixed precision
    tf32=True,
    
    # Logging & Saving
    logging_steps=50,
    save_steps=500,
    save_total_limit=3,
    evaluation_strategy="steps",
    eval_steps=500,
    
    # Other
    report_to="none",                    # Disable wandb/tensorboard
    remove_unused_columns=False,
    push_to_hub=False,
)

print("Training Configuration:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Mixed precision: {'BF16' if training_args.bf16 else 'FP16' if training_args.fp16 else 'FP32'}")

## 7. Initialize Trainer

In [ ]:
# Initialize SFT Trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_formatted,
    eval_dataset=val_dataset_formatted,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=2048,
    packing=False,  # Don't pack sequences (clearer for evaluation)
)

print("\n✅ Trainer initialized!")
print(f"   Training samples: {len(train_dataset_formatted):,}")
print(f"   Eval samples: {len(val_dataset_formatted):,}")
print(f"   Max sequence length: 2048 tokens")

## 8. Start Training

In [ ]:
# Clear CUDA cache
torch.cuda.empty_cache()

# Start training
print("\n🚀 Starting training...\n")
trainer.train()

print("\n✅ Training complete!")

## 9. Save Model

In [ ]:
# Save fine-tuned model
output_dir = "./smollm3-kmp-final"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Model saved to: {output_dir}")

# Optionally save to Google Drive
import shutil
drive_save_path = "/content/drive/MyDrive/smollm3-kmp-finetuned"
shutil.copytree(output_dir, drive_save_path, dirs_exist_ok=True)
print(f"✅ Model backed up to Google Drive: {drive_save_path}")

## 10. Test: Method Body Implementation Quality

**Critical Test**: Does the model generate FULL implementations or just signatures?

In [ ]:
def generate_code(prompt, model, tokenizer, max_new_tokens=512):
    """Generate code completion"""
    
    # Format prompt
    formatted_prompt = f"""<|im_start|>system
You are an expert Kotlin Multiplatform developer. Generate complete, working code with full method body implementations. Never output just signatures or TODO comments.<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant\n"""
    
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Extract only the assistant's response
    if "<|im_start|>assistant" in generated:
        response = generated.split("<|im_start|>assistant")[-1]
        response = response.replace("<|im_end|>", "").strip()
        return response
    
    return generated

print("✅ Generation function ready")

In [ ]:
def check_method_body_quality(code):
    """
    Analyze generated code to check if it has real method bodies.
    
    Returns:
        dict with quality metrics
    """
    results = {
        "has_braces": "{" in code and "}" in code,
        "has_return": "return " in code,
        "has_logic": False,
        "has_todo": "TODO" in code or "todo" in code.lower(),
        "has_empty_body": "{ }" in code or "{\n}" in code,
        "quality_score": 0,
    }
    
    # Check for real implementation logic
    logic_indicators = [
        "if (", "when (", "for (", "while (",
        ".map", ".filter", ".collect", ".launch",
        "try {", "catch", "emit(",
        "= ", "+=", "-=",
    ]
    
    logic_count = sum(1 for indicator in logic_indicators if indicator in code)
    results["has_logic"] = logic_count >= 2
    
    # Calculate quality score
    score = 0
    if results["has_braces"]: score += 1
    if results["has_return"]: score += 1
    if results["has_logic"]: score += 2
    if not results["has_todo"]: score += 1
    if not results["has_empty_body"]: score += 1
    
    results["quality_score"] = score
    results["is_full_implementation"] = score >= 4
    
    return results

print("✅ Quality checker ready")

### Test Case 1: Interface Implementation

In [ ]:
test_prompt_1 = """// Source set: commonMain
// Implement this interface with full method bodies:
interface UserRepository {
    suspend fun getUser(id: String): User
    suspend fun saveUser(user: User)
}"""

print("🧪 Test 1: Interface → Implementation\n")
print("INPUT:")
print(test_prompt_1)
print("\n" + "="*60 + "\n")

generated = generate_code(test_prompt_1, trainer.model, tokenizer, max_new_tokens=512)
print("GENERATED:")
print(generated)
print("\n" + "="*60 + "\n")

quality = check_method_body_quality(generated)
print("QUALITY ANALYSIS:")
print(f"  Has braces: {quality['has_braces']}")
print(f"  Has return statements: {quality['has_return']}")
print(f"  Has logic (if/when/etc): {quality['has_logic']}")
print(f"  Has TODO: {quality['has_todo']}")
print(f"  Has empty body: {quality['has_empty_body']}")
print(f"  Quality score: {quality['quality_score']}/6")
print(f"  ✅ Full implementation: {quality['is_full_implementation']}")

### Test Case 2: Expect/Actual Implementation

In [ ]:
test_prompt_2 = """// Source set: androidMain
// Implement the actual for this expect declaration:
expect class PlatformLogger() {
    fun log(message: String)
}"""

print("🧪 Test 2: Expect → Actual\n")
print("INPUT:")
print(test_prompt_2)
print("\n" + "="*60 + "\n")

generated = generate_code(test_prompt_2, trainer.model, tokenizer, max_new_tokens=512)
print("GENERATED:")
print(generated)
print("\n" + "="*60 + "\n")

quality = check_method_body_quality(generated)
print("QUALITY ANALYSIS:")
print(f"  Quality score: {quality['quality_score']}/6")
print(f"  ✅ Full implementation: {quality['is_full_implementation']}")

### Test Case 3: ViewModel with State Management

In [ ]:
test_prompt_3 = """// Source set: commonMain
// Implement ViewModel 'LoginViewModel' with state management, coroutines, and event handling

class LoginViewModel"""

print("🧪 Test 3: ViewModel Implementation\n")
print("INPUT:")
print(test_prompt_3)
print("\n" + "="*60 + "\n")

generated = generate_code(test_prompt_3, trainer.model, tokenizer, max_new_tokens=800)
print("GENERATED:")
print(generated)
print("\n" + "="*60 + "\n")

quality = check_method_body_quality(generated)
print("QUALITY ANALYSIS:")
print(f"  Quality score: {quality['quality_score']}/6")
print(f"  ✅ Full implementation: {quality['is_full_implementation']}")

### Test Case 4: Composable UI Function

In [ ]:
test_prompt_4 = """// Source set: commonMain
// Implement @Composable function 'LoginScreen' with full UI layout:

@Composable
fun LoginScreen()"""

print("🧪 Test 4: Composable UI\n")
print("INPUT:")
print(test_prompt_4)
print("\n" + "="*60 + "\n")

generated = generate_code(test_prompt_4, trainer.model, tokenizer, max_new_tokens=800)
print("GENERATED:")
print(generated)
print("\n" + "="*60 + "\n")

quality = check_method_body_quality(generated)
print("QUALITY ANALYSIS:")
print(f"  Quality score: {quality['quality_score']}/6")
print(f"  ✅ Full implementation: {quality['is_full_implementation']}")

## 11. Batch Evaluation on Test Set

In [ ]:
# Load test set
test_file = f"{DATA_DIR}/test.jsonl"
test_data = []
with open(test_file, 'r') as f:
    for line in f:
        test_data.append(json.loads(line))

print(f"Test set: {len(test_data):,} examples")

# Sample for evaluation (use first 50 for speed)
test_sample = test_data[:50]

In [ ]:
# Evaluate on test set
from tqdm.auto import tqdm

results = []

print("\n🔄 Evaluating on test set...\n")

for i, example in enumerate(tqdm(test_sample, desc="Testing")):
    input_text = example['input_text']
    expected = example['target_text']
    
    # Generate
    generated = generate_code(input_text, trainer.model, tokenizer, max_new_tokens=512)
    
    # Check quality
    quality = check_method_body_quality(generated)
    expected_quality = check_method_body_quality(expected)
    
    results.append({
        'pair_type': example.get('pair_type', 'unknown'),
        'generated_quality': quality['quality_score'],
        'expected_quality': expected_quality['quality_score'],
        'is_full_impl': quality['is_full_implementation'],
        'has_logic': quality['has_logic'],
        'has_todo': quality['has_todo'],
    })

print("\n✅ Evaluation complete!")

In [ ]:
# Analyze results
import pandas as pd
import numpy as np

df = pd.DataFrame(results)

print("\n" + "="*60)
print("📊 EVALUATION RESULTS")
print("="*60)

print(f"\nTotal samples evaluated: {len(df)}")

print(f"\n🎯 Implementation Quality:")
print(f"  Full implementations: {df['is_full_impl'].sum()} / {len(df)} ({100*df['is_full_impl'].mean():.1f}%)")
print(f"  Has logic (if/when/etc): {df['has_logic'].sum()} / {len(df)} ({100*df['has_logic'].mean():.1f}%)")
print(f"  Has TODO markers: {df['has_todo'].sum()} / {len(df)} ({100*df['has_todo'].mean():.1f}%)")

print(f"\n📈 Quality Scores:")
print(f"  Mean generated quality: {df['generated_quality'].mean():.2f} / 6")
print(f"  Mean expected quality:  {df['expected_quality'].mean():.2f} / 6")
print(f"  Quality retention:      {100*df['generated_quality'].mean()/df['expected_quality'].mean():.1f}%")

print(f"\n📋 By Pair Type:")
for ptype in df['pair_type'].unique():
    subset = df[df['pair_type'] == ptype]
    full_impl_pct = 100 * subset['is_full_impl'].mean()
    avg_quality = subset['generated_quality'].mean()
    print(f"  {ptype:30s} Full: {full_impl_pct:5.1f}%  Quality: {avg_quality:.2f}/6")

# Success criteria
print("\n" + "="*60)
full_impl_rate = 100 * df['is_full_impl'].mean()
if full_impl_rate >= 70:
    print("✅ SUCCESS: Model generates full implementations (≥70%)")
elif full_impl_rate >= 50:
    print("⚠️  PARTIAL: Model sometimes generates full implementations (50-70%)")
else:
    print("❌ FAILURE: Model mostly generates signatures/stubs (<50%)")
print("="*60)

## 12. Summary & Next Steps

In [ ]:
print("\n" + "="*60)
print("📋 TRAINING SUMMARY")
print("="*60)

print(f"\n✅ Training Complete")
print(f"   Model: {MODEL_NAME}")
print(f"   Training samples: {len(train_dataset):,}")
print(f"   Validation samples: {len(val_dataset):,}")
print(f"   Epochs: {training_args.num_train_epochs}")

print(f"\n💾 Model Saved")
print(f"   Local: ./smollm3-kmp-final")
print(f"   Drive: /content/drive/MyDrive/smollm3-kmp-finetuned")

if 'df' in globals():
    print(f"\n🎯 Evaluation Results")
    print(f"   Full implementations: {100*df['is_full_impl'].mean():.1f}%")
    print(f"   Average quality: {df['generated_quality'].mean():.2f}/6")

print("\n📝 Next Steps:")
print("   1. Test on your own KMP code examples")
print("   2. Integrate with your IDE")
print("   3. Fine-tune further if needed")
print("   4. Deploy to production")
print("="*60)